In [1]:
%%writefile Dockerfile
FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-inference:1.10.2-transformers4.17.0-gpu-py38-cu113-ubuntu20.04
RUN pip install --upgrade 'transformers==4.24.0'

Writing Dockerfile


In [3]:
%cd ~/SageMaker/sm-extend-container

/home/ec2-user/SageMaker/sm-extend-container


In [6]:
%%sh

# Specify an algorithm name
algorithm_name=transformers-extended-container

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Log into Docker
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon  114.7kB
Step 1/2 : FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-inference:1.10.2-transformers4.17.0-gpu-py38-cu113-ubuntu20.04
 ---> 27b277000343
Step 2/2 : RUN pip install --upgrade 'transformers==4.24.0'
 ---> Using cache
 ---> d908d2ab5cab
Successfully built d908d2ab5cab
Successfully tagged transformers-extended-container:latest
The push refers to repository [905847418383.dkr.ecr.us-east-1.amazonaws.com/transformers-extended-container]
4be8f41987db: Preparing
3ba6fe4b2b66: Preparing
b0134fef65ae: Preparing
1bfa98cef2c8: Preparing
b8bcd343d9df: Preparing
83e6952e8e57: Preparing
721041a08a77: Preparing
ca548dd9f6b6: Preparing
ac0871e16dd1: Preparing
981540c7bf62: Preparing
57c2137bc54a: Preparing
553f00fe9414: Preparing
c290b09be722: Preparing
2e36dd432e28: Preparing
335dafe074cd: Preparing
8480a7f24d40: Preparing
ca548dd9f6b6: Waiting
a688b15f64fd: Preparing
83e6952e8e57: Waiting
721041a08a77: Waiting
54

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

